In [ ]:
# to prevent to restart kernel when any changes are made to any imported file
%reload_ext autoreload
%autoreload 2

# to import any file from some other directory
# sys.path.append("/tmp/fastai/old")

# to stop printing warnings
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
warnings.warn = warn

import json,pdb,string,re
import pandas as pd
from os import walk
from tqdm import tqdm
tqdm.pandas()

from pathlib import Path

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_colwidth', -1)

In [ ]:
DIR='../MedcareLLC'

catalog_dir=f'{DIR}/catalog'
helium_dir=f'{DIR}/he_all'
keepa_dir=f'{DIR}/keepa_all'
output_dir=f'{DIR}/output'

Path(f'{DIR}/output').mkdir(exist_ok=True)

In [ ]:
def read_multiple_files(dir: str, kw: str):
    all_filenames = next(walk(f'{dir}'), (None, None, []))[2] 
    filenames_wanted=[f for f in all_filenames if kw.lower() in f.lower()]
    df=pd.DataFrame()
    for fn in filenames_wanted:
        tmpdf=pd.read_csv(f'{dir}/{fn}') if '.csv' in fn else pd.read_excel(f'{dir}/{fn}')

        # tmpdf=pd.read_csv(f'{dir}/{fn}')
        df=pd.concat([df, tmpdf])
    return df


def get_pack_size(row):
    size=row.Size.lower().strip()
    title=row.Title.lower().strip()

    size.translate(str.maketrans('', '', string.punctuation))
    title.translate(str.maketrans('', '', string.punctuation))

    product_desc=size+' '+title

    checks=[
        'pack of ([0-9]+)',
        '([0-9]+) pack',
        '([0-9]+) per pack',
        '([0-9]+)pack',
        '([0-9]+)-pack',
        '([0-9]+) ct pack',
        '([0-9]+) count pack',
    ]
    row['Pack Size']=1 # default value 1
    for chk in checks:
        sz=re.findall(chk, product_desc)
        
        if sz: row['Pack Size']=int(max(sz))

    return row
    

In [ ]:
catalog=read_multiple_files(catalog_dir, 'catalog')
print(catalog.shape)

catalog.head(1)

In [ ]:
# each_price
# upc_code
# missing pack size values
# check dtype of:
    # each_price
    # Pack Size	
    # fill missing buy box prices
# add check for 'Set of' along with Pack of
# missing BuyBox 30 day avg values: create func that gets Product price by priority on Price columns
# compare whose new offer count, seller type is better keepa Vs jscout

rename_columns_catalog=['Title']
rename_dict = dict(zip(rename_columns_catalog, [f'{x}_catalog' for x in rename_columns_catalog]))
catalog = catalog.rename(columns=rename_dict)


catalog_asin_code = ''
catalog_upc_code  = 'UPC'
catalog_each_price= 'Base Price'
catalog_product_name='Title_catalog'





keepa_asin_code='ASIN'
keepa_upc_code='Product Codes: UPC'

# we give priority to asin column, which is used to merge with keepa_scanned file
catalog_id_column=catalog_asin_code if catalog_asin_code else catalog_upc_code
keepa_id_column= keepa_asin_code if catalog_asin_code else keepa_upc_code

catalog[catalog_id_column]=catalog[catalog_id_column].astype(str)


catalog_id_column, keepa_id_column


# Special Filters

In [ ]:
catalog['Base Price']=catalog['Base Price'].str.replace('$','', regex=False).str.replace(',','', regex=False).astype(float)
catalog.head()

In [ ]:
# catalog['Case pack price']=catalog['Case size']*catalog['Base Price']

# catalog=catalog[(catalog['Case pack price']<=250.0)&(catalog['Base Price']>=5)&(catalog['Base Price']<=50)]
# catalog.shape

# 1. Catalog

In [ ]:
print(catalog[catalog_asin_code].isnull().sum()) if catalog_asin_code else print('missing ASIN column in catalog')

In [ ]:
print(catalog[catalog_upc_code].isnull().sum()) if catalog_upc_code else print('missing UPC column in catalog')

In [ ]:
print('Search on keepa using: ', catalog_id_column, catalog[catalog_upc_code].nunique())


In [ ]:
# with open ('ids.txt', 'w') as f:
#     f.write(list(catalog[catalog_id_column].unique()))

print(','.join(list(catalog[catalog_id_column].unique())))

# 2. Keepa Scanned

In [ ]:
keepa_cols=['ASIN','URL: Amazon','Amazon: Availability of the Amazon offer', 'Product Codes: UPC', 'Size', 'Title']

In [ ]:
keepa=read_multiple_files(keepa_dir, 'keepa')
keepa['Product Codes: UPC']=keepa['Product Codes: UPC'].astype(str)
print(keepa.shape)

keepa.head(1)


In [ ]:
keepa['Product Codes: UPC'].fillna('', inplace=True)
keepa['Product Codes: UPC'] = keepa['Product Codes: UPC'].str.split(', ')
keepa=keepa.explode('Product Codes: UPC')
keepa.shape

# 3. Merge (Keepa + Catalog)

In [ ]:
catalog[catalog_id_column].nunique(), keepa[keepa_id_column].nunique()

In [ ]:
keepa_merged=pd.merge(catalog, keepa, how='inner', left_on=catalog_id_column, right_on=keepa_id_column)
keepa_merged.drop(keepa_merged.loc[:,keepa_merged.columns.str.contains('Unnamed')].columns.tolist(), axis=1, inplace=True)

print(keepa_merged.shape, keepa_merged.ASIN.nunique())

keepa_merged.head(1)

# 4. Helium

In [ ]:
def get_chunks(asins, chunk_size=350):
    chunks=[]
    for i in range(0, len(asins), chunk_size):
        chunk = asins[i:i+chunk_size]
        chunks.append(chunk)
        print('|'.join(chunk), '\n\n')
    # return chunksb

In [ ]:
asins=keepa_merged.ASIN.unique().tolist()
print(len(asins))
get_chunks(asins, 350)


In [ ]:
helium=read_multiple_files(helium_dir, 'helium')
helium['Price $']=helium['Price $'].str.replace(',','').astype(float)
print(helium.shape)

helium.head(1)

In [ ]:
helium['Sales']=helium['Sales'].astype(str).str.replace(',','').astype(float)
helium['Revenue']=helium['Revenue'].astype(str).str.replace(',','').astype(float)
helium['Review Count']=helium['Review Count'].astype(str).str.replace(',','').astype(float)



In [ ]:
print(f'Missing Helium Price-$ values: {round((helium["Price $"].isnull().sum()/helium.shape[0])*100, 2)}%')



# 5. Merge All [(Keepa+Catalog) + Helium]

In [ ]:
merged=pd.merge(keepa_merged, helium, on='ASIN', how='inner')
merged=merged.drop_duplicates(['ASIN'])
print(merged.shape)
merged.head(1)

In [ ]:
merged['Size']=merged['Size'].astype(str).fillna('1')
merged=merged.progress_apply(get_pack_size, axis=1)
merged[catalog_each_price]=merged[catalog_each_price].astype(str).str.strip().str.replace('$', '', regex=False).astype(float)
merged['Pack Size']=merged['Pack Size'].astype(float)


In [ ]:
# filling missing values of helium Price$ with keepa lowest list price
merged['Price $']=merged['Price $'].fillna(merged['List Price: Lowest'])
print(f'Missing Helium Price-$ values: {round((merged["Price $"].isnull().sum()/merged.shape[0])*100, 2)}%')

# 6. Filtering

### 6.1. Soft Filter

In [ ]:
merged=merged[(merged['Amazon: Availability of the Amazon offer']=='no Amazon offer exists') & 
              ( merged['Buy Box'] != 'Amazon' )]
merged=merged[(merged['Ratings']>=4.0)]
merged=merged[(merged['Review Count']>=20)]
merged=merged[(merged['Active Sellers #']>=3)]
merged=merged[merged['Size Tier'].isin(['Large Standard-Size', 'Small Standard-Size']) ] # from helium 

### 6.2. Create Custom Columns

In [ ]:
merged['purchase_price']=merged['Pack Size']*merged[catalog_each_price]
merged['net_profit']=merged['Price $'] - merged['purchase_price'] - merged['FBA Fees $']
merged['net_profit_%']=(merged['net_profit']/merged['purchase_price'])*100

merged['expected_sales']=merged['Sales']/merged['Active Sellers #']
merged['Invest_monthly']=merged['purchase_price']*merged['expected_sales']
merged['ROI_monthly']=merged['net_profit']*merged['expected_sales']
merged['ROI_monthly_%']=(merged['ROI_monthly']/merged['Invest_monthly'])*100

final=merged[merged['net_profit_%']>=0]
print(final.shape)

In [ ]:
from sys import hash_info

profitable=final.sort_values(['net_profit_%'], ascending=False)[[f'{keepa_asin_code}', f'{catalog_product_name}', f'{keepa_upc_code}', 'URL: Amazon', 'Pack Size', f'{catalog_each_price}', 'Price $', 'FBA Fees $', 'net_profit',	'net_profit_%'	,'expected_sales',	'Invest_monthly',	'ROI_monthly'	,'ROI_monthly_%']]

print(profitable.shape)

# B007X6WHVE, B01BH6SYXU, B0018QE4IE, B00HSYJRNM, B00EV5BY3M, B07RD16F1W           ### --usman
# ((B08W5DTJQR,Sellers=2), B000LM4EOA, B000GX4AA2, B000FRVA8O, BBox=Amazon/Helium) ### -- faisal

# 4 asins jo meri sheet main nhi hash_info
# 3: buy box amazon as per helium 
# 1: seller 


In [ ]:
profitable[profitable['expected_sales']>=30].shape

In [ ]:
winners=profitable[(profitable['net_profit_%']>=25)&(profitable['expected_sales']>=30)].drop_duplicates(['ASIN'])
print(winners.shape)
winners

In [ ]:
winners.to_excel(f'{output_dir}/medcare_winners.xlsx')                              
